<a href="https://colab.research.google.com/github/yorion01/beginning/blob/main/%EC%A3%BC%EA%B0%80%EC%98%88%EC%B8%A1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import scipy.stats as stats
import math
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from torch.utils.data import DataLoader, Dataset

import torch

In [ ]:
df = pd.read_excel('/content/금융데이터_전처리.xlsx')

In [ ]:
df = df.set_index('Date')

### 데이터 분리

In [ ]:
# 특성
feature1_list = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','log_return']
feature2_list = ['MA5', 'MA10', 'RASD5', 'RASD10', 'MACD', 'CCI', 'ATR', 'ub', 'middle', 'lb', 'MTM1', 'MTM3', 'ROC', 'WPR',]
feature3_list = ['S&P500', 'SOX', 'VIX']
feature4_list = ['next_rtn']

all_feature = feature1_list + feature2_list + feature3_list + feature4_list

phase_flage = '3'

if phase_flage == '1':
    train_form = datetime(2010,1,4)
    train_to = datetime(2012,1,1)
    val_form = datetime(2012,1,1)
    val_to = datetime(2012,4,1)
    test_form = datetime(2010,4,1)
    test_to = datetime(2012,7,1)

elif phase_flage == '2':
    train_form = datetime(2012,7,1)
    train_to = datetime(2014,7,1)
    val_form = datetime(2014,7,1)
    val_to = datetime(2014,10,1)
    test_form = datetime(2014,10,1)
    test_to = datetime(2015,1,1)

else:
    train_form = datetime(2015,1,1)
    train_to = datetime(2017,1,1)
    val_form = datetime(2017,1,1)
    val_to = datetime(2017,4,1)
    test_form = datetime(2017,4,1)
    test_to = datetime(2017,7,1)

# 학습/검증/테스트
train_df = df.loc[train_form:train_to,all_feature].copy()
val_df = df.loc[val_form:val_to,all_feature].copy()
test_df = df.loc[test_form:test_to,all_feature].copy()

### 정규화

In [ ]:
def min_max_normal(tmp_df):
    eng_list = []
    sample_df = tmp_df.copy()
    for x in all_feature:  # x는 각 col 중 한개
        if x in feature4_list:
            continue
        series = sample_df[x].copy()
        values = series.values   # 1*n
        values = values.reshape((len(values),1))  # n*1

        # 스케일러 생성 및 훈련
        scaler = MinMaxScaler(feature_range=(0,1)) # 최소 0, 최대 1
        scaler = scaler.fit(values)

        # 데이터셋 정규화 및 출력
        normalized = scaler.transform(values)
        new_feature = '{}_normal'.format(x)
        eng_list.append(new_feature)
        sample_df[new_feature] = normalized
    return sample_df,eng_list

In [ ]:
train_sample_df , eng_list = min_max_normal(train_df)
val_sample_df , eng_list = min_max_normal(val_df)
test_sample_df , eng_list = min_max_normal(test_df)

In [ ]:
# # 학습 데이터와 레이블 데이터 나눔
# def create_dataset_binary(data, feature_list, step, n):    # step은 몇개를 한번에 묶어서 학습할 것인가, n은 총 col수-1(1개는 결과값)
#     # LSTM에 넣어줄 변수 데이터 선택 - 배열로 만듬    // 데이터 수*n
#     train_xdata = np.array(data[feature_list[0:n]])
#     # m = 총 단계 개수 Ex. [0,1,2,,,,,,,,m]
#     m = np.arange(len(train_xdata)-step)
#     x,y = [], []
#     for i in m:
#         # 각 단계마다 사용할 학습 데이터 기간 설정
#         a = train_xdata[i:(i+step)]
#         x.append(a)
#     # 신경망에 사용할 데이터 정리(3차원으로 구성)
#     x_batch = np.reshape(np.array(x), (len(m),step,n))  # m*step*n

#     # 레이블 데이터 생성(종가)
#     train_ydata = np.array(data[[feature_list[n]]])

#     # n_step 이상부터 답을 사용할 수 있다.
#     for i in m + step:
#         start_price = train_ydata[i-1][0]     # 전날 종가
#         end_price = train_ydata[i][0]     # 당일 종가

#         if end_price>start_price:
#             label = 1
#         else:
#             label = 0
#         y.append(label)
#     y_batch = np.reshape(np.array(y), (-1,1))
#     return torch.FloatTensor(x_batch).to(device), torch.FloatTensor(y_batch).to(device)

# num_step = 5
# num_unit = 200
# n_feature = len(eng_list)-1

# x_train, y_train = create_dataset_binary(train_sample_df[eng_list], eng_list, num_step, n_feature)
# x_val, y_val = create_dataset_binary(val_sample_df[eng_list], eng_list, num_step, n_feature)
# x_test, y_test = create_dataset_binary(test_sample_df[eng_list], eng_list, num_step, n_feature)

# train = torch.utils.data.TensorDataset(x_train, y_train)
# val = torch.utils.data.TensorDataset(x_val, y_val)
# test = torch.utils.data.TensorDataset(x_test, y_test)

# batch_size = 10
# train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
# val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

In [ ]:
len(train_sample_df)

504

In [ ]:
num_step = 5
num_unit = 200
n_feature = len(eng_list)-1

class windowDataset(Dataset):
    def __init__(self, data, feature_list = eng_list, step = num_step, n = n_feature):
        self.feature_list = feature_list
        train_xdata = np.array(data[self.feature_list[0:n]])  # 504 * 23
        # m = 총 단계 개수 Ex. [0,1,2,,,,,,,,m]
        m = np.arange(len(train_xdata)-step)   # (499,)
        x,y = [], []
        for i in m:
            # 각 단계마다 사용할 학습 데이터 기간 설정
            a = train_xdata[i:(i+step)]
            x.append(a)
        # 신경망에 사용할 데이터 정리(3차원으로 구성)
        self.x = torch.FloatTensor(np.reshape(np.array(x), (len(m),step,n)))  # 499*5*23 -> 499 * 5 * 23

        # 레이블 데이터 생성(종가)
        train_ydata = np.array(data[[self.feature_list[n]]])

        # n_step 이상부터 답을 사용할 수 있다.
        for i in m + step:
            start_price = train_ydata[i-1][0]     # 전날 종가
            end_price = train_ydata[i][0]     # 당일 종가

            if end_price>start_price:
                label = 1
            else:
                label = 0
            y.append(label)
        self.y = torch.FloatTensor(np.reshape(np.array(y), (-1,1)))
        self.len = len(self.x)

    # 이거 갯수만큼 돌게 됨 -> 이거 갯수 틀리면 학습시 에러뜸!!
    def __len__(self):
        return self.len

    # return 값들
    def __getitem__(self, i):
        return self.x[i], self.y[i]



In [ ]:
train_dataset = windowDataset(train_sample_df)
train_loader = DataLoader(train_dataset, batch_size=10, drop_last=True)
val_dataset = windowDataset(val_sample_df)
val_loader = DataLoader(val_dataset, batch_size=10, drop_last=True)
test_dataset = windowDataset(test_sample_df)
test_loader = DataLoader(test_dataset, batch_size=10, drop_last=True)

In [ ]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape[1:]))
  print("y 크기 : {}".format(y.shape[1:]))
  break

X 크기 : torch.Size([5, 23])
y 크기 : torch.Size([1])


In [ ]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cpu is available


### 모델 생성하기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
input_size = 23
num_layers = 2
hidden_size = 200
sequence_length = 5

In [ ]:
class LSTM_model(nn.Module):

  def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
    super(LSTM_model, self).__init__()
    self.device = device
    self.hidden_size = hidden_size   # 5
    self.num_layers = num_layers     # 5
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  # batch_size * 5(일수) * 23(column 수) -> batch_size * 5 * 5
    self.fc = nn.Sequential(nn.Linear(hidden_size * sequence_length, 1), nn.Sigmoid())

  # x : 20(batch_size) * 5 * 23,
  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device) # 초기 hidden state 설정하기. (layer 높이, 배치사이즈, )  num_layers * batch_size * hidden_size
    c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device) # 이거 추가 -> rnn과 다른점  num_layers * batch_size * hidden_size
    out, hidden = self.lstm(x, (h0,c0)) # out: RNN의 마지막 레이어로부터 나온 output feature 를 반환한다. hn: hidden state를 반환한다.
    out = out.reshape(out.shape[0], -1) # many to many 전략
    out = self.fc(out)  # 이걸 넣으면 many to one 됨
    return out


model = LSTM_model(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device).to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,5,23))
model(basic_data)

작동하는지 실험


tensor([[0.4965]], grad_fn=<SigmoidBackward0>)

In [ ]:
class LSTM_model(nn.Module):

  def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
    super(LSTM_model, self).__init__()
    self.device = device
    self.hidden_size = hidden_size   # 200
    self.num_layers = num_layers     # 5
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  # batch_size * 5(일수) * 23(column 수) -> batch_size * 5 * 200
    self.lstm1 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
    self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
    self.lstm3 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
    self.lstm4 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
    self.batch = nn.BatchNorm1d(5)
    self.dropout = nn.Dropout(p=0.25)
    self.fc = nn.Sequential(nn.Linear(hidden_size * sequence_length, 1), nn.Sigmoid())

  # x : 20(batch_size) * 5 * 23,
  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device) # 초기 hidden state 설정하기. (layer 높이, 배치사이즈, )  5 * batch_size * 5
    c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device) # 이거 추가 -> rnn과 다른점  5 * batch_size * 5
    out0, hidden = self.lstm(x, (h0,c0)) # out: RNN의 마지막 레이어로부터 나온 output feature 를 반환한다. hn: hidden state를 반환한다.
    out0 = self.batch(out0)

    for i in range(1,5):
      globals()['h{}'.format(i)] = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
      globals()['c{}'.format(i)] = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)

    out1, _ = self.lstm1(out0,(h1,c1))
    out1 = self.dropout(out1)
    out2, _ = self.lstm2(out1,(h2,c2))
    out2 = self.batch(out2)
    out3, _ = self.lstm3(out2,(h3,c3))
    out3 = self.dropout(out3)
    out4, _ = self.lstm4(out3,(h4,c4))

    out = out4.reshape(out4.shape[0], -1) # many to many 전략
    out = self.fc(out)
    return out


model = LSTM_model(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device).to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,5,23))
model(basic_data)


작동하는지 실험


tensor([[0.5060]], grad_fn=<SigmoidBackward0>)

In [ ]:
model

LSTM_model(
  (lstm): LSTM(23, 200, num_layers=2, batch_first=True)
  (lstm1): LSTM(200, 200, num_layers=2, batch_first=True)
  (lstm2): LSTM(200, 200, num_layers=2, batch_first=True)
  (lstm3): LSTM(200, 200, num_layers=2, batch_first=True)
  (lstm4): LSTM(200, 200, num_layers=2, batch_first=True)
  (batch): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=1000, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [ ]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=lr)

### 모델 학습하기

In [ ]:
loss_graph = [] # 그래프 그릴 목적인 loss.
n = len(train_loader)
best_val_loss = np.inf
best_val_acc = 0

for epoch in range(num_epochs):
    running_loss = 0.0
    num = 0
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함

              inputs = inputs.to(device)
              labels = labels.to(device)
              out = model(inputs)
              loss = criterion(out, labels)

              optimizer.zero_grad() #
              loss.backward() # loss가 최소가 되게하는
              optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
              running_loss += loss.item() # 한 배치의 loss 더해주고,
    print("train")
    print(running_loss)
    with torch.no_grad():
              print("Calculating validation results...")
              model.eval()
              val_loss_items = []
              val_acc_items = []
              figure = None
              for inputs, labels in val_loader:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outs = model(inputs)
                  preds = torch.argmax(outs, dim=-1)

                  loss_item = criterion(outs, labels).item()
                  acc_item = (labels == preds).sum().item()
                  val_loss_items.append(loss_item)


              val_loss = np.sum(val_loss_items)
              val_acc = np.sum(val_acc_items) / len(val_loader)
              best_val_loss = min(best_val_loss, val_loss)
              # if val_acc > best_val_acc:
              #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
              #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
              #     best_val_acc = val_acc
              # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
              print("val")
              print(
                  f"loss: {val_loss:4.8} || "
                  f"best loss: {best_val_loss:4.2}"
              )

train
12.282976388931274
Calculating validation results...
val
loss: 1.2521369 || best loss:  1.3
train
12.254771769046783
Calculating validation results...
val
loss: 1.2516711 || best loss:  1.3
train
12.248005673289299
Calculating validation results...
val
loss: 1.2517587 || best loss:  1.3
train
12.244600385427475
Calculating validation results...
val
loss: 1.2517434 || best loss:  1.3
train
12.24311175942421
Calculating validation results...
val
loss: 1.2518128 || best loss:  1.3
